# Paramtypes explained

Internally in the SQLite database on disk, data are registered as being of one of (currently) three allowed types:

- `numeric`
- `array`
- `text`

This notebook seeks to exemplify when each type should be used, and how the `Measurement` object treats data of each type differently.

In [ ]:
import os

import numpy as np

from qcodes.instrument.parameter import ArrayParameter, Parameter
from qcodes.dataset.database import initialise_or_create_database_at
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.data_set import load_by_id
from qcodes.dataset.data_export import get_data_by_id
from qcodes.tests.instrument_mocks import DummyInstrument
import time

In [2]:
initialise_or_create_database_at(os.path.join(os.getcwd(), 'paramtypes_explained.db'))
new_experiment('paramtypes', sample_name='not_available')

paramtypes#not_available#8@C:\Users\wihpniel\src\Qcodes\docs\examples\DataSet\paramtypes_explained.db
-----------------------------------------------------------------------------------------------------

In [3]:
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
SA = DummyInstrument('SA')

In [4]:
# some array-like data types

class Spectrum(ArrayParameter):
    
    def __init__(self, name, instrument):
        
        self.N = 7
        setpoints = (np.linspace(0, 1, self.N),)
    
        super().__init__(name=name,
                         instrument=instrument,
                         setpoints=setpoints,
                         shape=(20,),
                         label='Noisy spectrum',
                         unit='V/sqrt(Hz)',
                         setpoint_names=('Frequency',),
                         setpoint_units=('Hz',))
        
    def get_raw(self):
        return np.random.randn(self.N)
    

class MultiDimSpectrum(ArrayParameter):
    
    def __init__(self, name, instrument):
        self.start = 0
        self.stop = 1
        self.npts = (2, 5, 3)
        sp1 = np.linspace(self.start, self.stop,
                          self.npts[0])
        sp2 = np.linspace(self.start, self.stop,
                          self.npts[1])
        sp3 = np.linspace(self.start, self.stop,
                          self.npts[2])
        setpoints = (sp1,
                     np.tile(sp2, (len(sp1), 1)),
                     np.tile(sp3, (len(sp1), len(sp2), 1)))
        super().__init__(name=name,
                         instrument=instrument,
                         setpoints=setpoints,
                         shape=(100, 50, 20),
                         label='Flower Power Spectrum in 3D',
                         unit='V/sqrt(Hz)',
                         setpoint_names=('Frequency0', 'Frequency1',
                                         'Frequency2'),
                             setpoint_units=('Hz', 'Other Hz', "Third Hz"))
    def get_raw(self):
        a = self.npts[0]
        b = self.npts[1]
        c = self.npts[2]
        return np.reshape(np.arange(a*b*c), (a, b, c))
    
# a string-valued parameter
def dac1_too_high():
    return 'Too high' if dac.ch1() > 5 else 'OK'

In [5]:
dac.add_parameter('control', get_cmd=dac1_too_high)
SA.add_parameter('spectrum', parameter_class=Spectrum)
SA.add_parameter('spectrum3D', parameter_class=MultiDimSpectrum)

## Numeric

The `numeric` datatype is simply a number. Data registered with this type are saved as individual numbers. This is the **default** datatype when registering parameters.

### Numeric example 1

In this example, all parameters get registered as `numeric` type. This entails that the array in unraveled and inserted point-by-point.

In [6]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum, setpoints=(dac.ch1,))

t0 = time.perf_counter()

with meas.run() as datasaver:
    for dac_v in np.linspace(0, 2, 5):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (SA.spectrum, SA.spectrum()))

t1 = time.perf_counter()

print(f'Finished run in {(t1-t0):.3f} s')
        
run_id1 = datasaver.run_id

Starting experimental run with id: 28
Finished run in 0.290 s


The data may be retrieved using `get_data_by_id`. This function will always bring back the data **as if** it had been stored in a point-by-point format. That is to say, all parameters come out as array of the same length, with setpoints values appropriately repeated.

In [7]:
get_data_by_id(run_id1)

[[{'name': 'dac_ch1',
   'label': 'Gate ch1',
   'unit': 'V',
   'data': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0.5,  0.5,
           0.5,  0.5,  0.5,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1.5,
           1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  2. ,  2. ,  2. ,  2. ,  2. ,
           2. ,  2. ])},
  {'name': 'SA_Frequency',
   'label': '',
   'unit': 'Hz',
   'data': array([ 0.        ,  0.16666667,  0.33333333,  0.5       ,  0.66666667,
           0.83333333,  1.        ,  0.        ,  0.16666667,  0.33333333,
           0.5       ,  0.66666667,  0.83333333,  1.        ,  0.        ,
           0.16666667,  0.33333333,  0.5       ,  0.66666667,  0.83333333,
           1.        ,  0.        ,  0.16666667,  0.33333333,  0.5       ,
           0.66666667,  0.83333333,  1.        ,  0.        ,  0.16666667,
           0.33333333,  0.5       ,  0.66666667,  0.83333333,  1.        ])},
  {'name': 'SA_spectrum',
   'label': 'Noisy spectrum',
   'unit': 'V/sqrt(Hz)',


## Array

The array paramtype stores data as binary blobs in the database. Insertion is faster (asymptotically **much** faster) this way, but the data are "dead" to SQL queries inside the database.

### Array example 1

Let us repeat the above measurement, but this time using `array` paramtypes.

In [8]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum, setpoints=(dac.ch1,), paramtype='array')

t0 = time.perf_counter()

with meas.run() as datasaver:
    for dac_v in np.linspace(0, 2, 5):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (SA.spectrum, SA.spectrum()))

t1 = time.perf_counter()

print(f'Finished run in {(t1-t0):.3f} s')
        
run_id2 = datasaver.run_id

Starting experimental run with id: 29
Finished run in 0.294 s


Reading back out the data, we can **not** see whether they were stored as `numeric` or `array` type.

In [9]:
get_data_by_id(run_id2)

[[{'name': 'dac_ch1',
   'label': 'Gate ch1',
   'unit': 'V',
   'data': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0.5,  0.5,
           0.5,  0.5,  0.5,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1.5,
           1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  2. ,  2. ,  2. ,  2. ,  2. ,
           2. ,  2. ])},
  {'name': 'SA_Frequency',
   'label': '',
   'unit': 'Hz',
   'data': array([ 0.        ,  0.16666667,  0.33333333,  0.5       ,  0.66666667,
           0.83333333,  1.        ,  0.        ,  0.16666667,  0.33333333,
           0.5       ,  0.66666667,  0.83333333,  1.        ,  0.        ,
           0.16666667,  0.33333333,  0.5       ,  0.66666667,  0.83333333,
           1.        ,  0.        ,  0.16666667,  0.33333333,  0.5       ,
           0.66666667,  0.83333333,  1.        ,  0.        ,  0.16666667,
           0.33333333,  0.5       ,  0.66666667,  0.83333333,  1.        ])},
  {'name': 'SA_spectrum',
   'label': 'Noisy spectrum',
   'unit': 'V/sqrt(Hz)',


### Array example 2

When storing multidimensional array data (think: Alazar cards), both `numeric` and `array` type can be used.

In [10]:
meas = Measurement()
meas.register_parameter(SA.spectrum3D, paramtype='array')

with meas.run() as datasaver:
    datasaver.add_result((SA.spectrum3D, SA.spectrum3D()))
run_id3 = datasaver.run_id

Starting experimental run with id: 30


The data come out the way we expect them to.

In [11]:
get_data_by_id(run_id3)

[[{'name': 'SA_Frequency0',
   'label': '',
   'unit': 'Hz',
   'data': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.])},
  {'name': 'SA_Frequency1',
   'label': '',
   'unit': 'Other Hz',
   'data': array([ 0.  ,  0.  ,  0.  ,  0.25,  0.25,  0.25,  0.5 ,  0.5 ,  0.5 ,
           0.75,  0.75,  0.75,  1.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,
           0.25,  0.25,  0.25,  0.5 ,  0.5 ,  0.5 ,  0.75,  0.75,  0.75,
           1.  ,  1.  ,  1.  ])},
  {'name': 'SA_Frequency2',
   'label': '',
   'unit': 'Third Hz',
   'data': array([ 0. ,  0.5,  1. ,  0. ,  0.5,  1. ,  0. ,  0.5,  1. ,  0. ,  0.5,
           1. ,  0. ,  0.5,  1. ,  0. ,  0.5,  1. ,  0. ,  0.5,  1. ,  0. ,
           0.5,  1. ,  0. ,  0.5,  1. ,  0. ,  0.5,  1. ])},
  {'name': 'SA_spectrum3D',
   'label': 'Flower Power Spectrum in 3D',
   'unit': 'V/sqrt(Hz)',
   'data': array([ 0,  1,  2,  3,  4

### Array example 3

For completeness, an example where the multidimensional array has an auxiliary setpoint.

In [12]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum3D, paramtype='array', setpoints=(dac.ch1,))

with meas.run() as datasaver:
    for dac_v in [3, 4, 5]:
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v),
                             (SA.spectrum3D, SA.spectrum3D()))
run_id4 = datasaver.run_id

Starting experimental run with id: 31


In [13]:
get_data_by_id(run_id4)

[[{'name': 'dac_ch1',
   'label': 'Gate ch1',
   'unit': 'V',
   'data': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
          4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
          5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])},
  {'name': 'SA_Frequency0',
   'label': '',
   'unit': 'Hz',
   'data': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])},
  {'name': 'SA_Frequency1

## Text

Text is strings. Sometimes it may be useful to capture categorial data that is represented as string values, or a log message, or else.

Note that the `paramtype` setting is important. The datasaver will not allow to save numeric data for a parameter that was registered as `'text'`. The opposite it also true: the datasaver will not allow to save strings for a parameter what was registered as non-`'text'` (`'numeric'` or `'array'`).

In [14]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.control, setpoints=(dac.ch1,), paramtype='text')

with meas.run() as datasaver:
    for dac_v in np.linspace(4, 6, 10):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v),
                             (dac.control, dac.control()))
run_id5 = datasaver.run_id

Starting experimental run with id: 32


In [15]:
get_data_by_id(run_id5)

[[{'name': 'dac_ch1',
   'label': 'Gate ch1',
   'unit': 'V',
   'data': array([ 4.        ,  4.22222222,  4.44444444,  4.66666667,  4.88888889,
           5.11111111,  5.33333333,  5.55555556,  5.77777778,  6.        ])},
  {'name': 'dac_control',
   'label': 'control',
   'unit': '',
   'data': array(['OK', 'OK', 'OK', 'OK', 'OK', 'Too high', 'Too high', 'Too high',
          'Too high', 'Too high'],
         dtype='<U8')}]]